# Hackathon Oil&Gas: Data Warehouse

## Participants
- Cindy Ortega - cindy.or03@gmail.com
- Nicolás Bueno - nbuenoz@unal.edu.co
- Alejandro Uribe - jduriber@unal.edu.co

## Libraries

In [1]:
import sqlite3
import numpy as np
import pandas as pd
from utils import dataProc
from utils import downloadData
import matplotlib.pyplot as plt
from pathlib import Path

## Downloading, Loading & Cleanning Data

In [2]:
# Scrappes and downloads the data from ANH website
download = downloadData.downloadData()
download.getData()

#Load Blind test data and Data
data = dataProc.dataProc()
df = data.loadData()
df_dict_blind = data.loadBlindData()
df_dict = data.cleanData(df)

Scrapping started
Scrapping Failed. Trying again
Scrapping started
Scrapping finished
['/Operaciones-Regalías-y-Participaciones/Sistema-Integrado-de-Operaciones/Documentos%20compartidos/Producción%20Fiscalizada%20Crudo%202020%20Agosto.xlsx', '/Operaciones-Regalías-y-Participaciones/Sistema-Integrado-de-Operaciones/Documentos%20compartidos/Producción%20Fiscalizada%20Crudo%202020%20Julio.xlsx', '/Operaciones-Regalías-y-Participaciones/Sistema-Integrado-de-Operaciones/Documents/Producción%20Fiscalizada%20Crudo%202020%20Junio.xlsx', '/Operaciones-Regalías-y-Participaciones/Sistema-Integrado-de-Operaciones/Documents/Producción%20Fiscalizada%20Crudo%202020%20Mayo.xlsx', '/Operaciones-Regalías-y-Participaciones/Sistema-Integrado-de-Operaciones/Documents/Producción%20Fiscalizada%20Crudo%202020%20Abril.xlsx', '/Operaciones-Regalías-y-Participaciones/Sistema-Integrado-de-Operaciones/Documents/Producción%20Fiscalizada%20Crudo%202020%20Marzo.xlsx', '/Operaciones-Regalías-y-Participaciones/Sistema-

TypeError: 'NoneType' object is not iterable

## Database
The data can be also read from a SQLite Database as follows.

In [ ]:
db_dir=Path('./database/anh_data.db')
conn = sqlite3.connect(db_dir)
c = conn.cursor()

The tables' names stored in the database can be retrieved as follows:

In [4]:
c.execute('''SELECT name FROM sqlite_master WHERE type ='table' AND name NOT LIKE 'sqlite_%';''').fetchall()

[('crude_2017',),
 ('crude_2016',),
 ('crude_2013',),
 ('crude_2019',),
 ('crude_2015',),
 ('crude_2014',),
 ('crude_2018',),
 ('crude_2020',)]

The `crude_2018` table's headers ca be retrieved as follows:

In [5]:
c.execute('''PRAGMA table_info(crude_2019)''').fetchall()

[(0, 'departamento', 'TEXT', 0, None, 0),
 (1, 'municipio', 'TEXT', 0, None, 0),
 (2, 'operadora', 'TEXT', 0, None, 0),
 (3, 'contrato', 'TEXT', 0, None, 0),
 (4, 'campo', 'TEXT', 0, None, 0),
 (5, 'enero', 'TEXT', 0, None, 0),
 (6, 'febrero', 'TEXT', 0, None, 0),
 (7, 'marzo', 'TEXT', 0, None, 0),
 (8, 'abril', 'TEXT', 0, None, 0),
 (9, 'mayo', 'TEXT', 0, None, 0),
 (10, 'junio', 'TEXT', 0, None, 0),
 (11, 'julio', 'TEXT', 0, None, 0),
 (12, 'agosto', 'TEXT', 0, None, 0),
 (13, 'septiembre', 'TEXT', 0, None, 0),
 (14, 'octubre', 'TEXT', 0, None, 0),
 (15, 'noviembre', 'TEXT', 0, None, 0),
 (16, 'diciembre', 'TEXT', 0, None, 0)]

Other queries are shown below:

In [6]:
c.execute('''SELECT * FROM crude_2018 LIMIT 1''').fetchall()

[('antioquia',
  'PUERTO NARE',
  'ecopetrol s.a.',
  'operacion-directa ecopetrol',
  'area teca-cocorna',
  '1290.88838709677',
  '1232.14785714286',
  '1146.54709677419',
  '1183.47233333333',
  '1264.54193548387',
  '1201.59633333333',
  '1276.36161290323',
  '1326.87032258065',
  '1404.01266666667',
  '1218.92483870968',
  '1333.641',
  '1373.87677419355')]

For example, the top 5 departamentos which produced crude in January 2018.

In [7]:
c.execute('''SELECT departamento, SUM(enero) AS total FROM crude_2018 GROUP BY crude_2018.departamento 
                ORDER BY total DESC LIMIT 5''').fetchall()

[('meta', 425024.3983870972),
 ('casanare', 162805.02645161285),
 ('santander', 62680.61967741938),
 ('arauca', 46571.40838709677),
 ('putumayo', 36501.45064516129)]

## Blind test

In [11]:
df_dict_blind['2017'].head()

,DEPARTAMENTO,MUNICIPIO,OPERADORA,CONTRATO,CAMPO,ENERO,FEBRERO,MARZO,ABRIL,MAYO,JUNIO,JULIO,AGOSTO,SEPTIEMBRE,OCTUBRE,NOVIEMBRE,DICIEMBRE,CUENCA,EMPRESA
0,cf33cb8a,cf33cb8a,d5580f74,76a16657,1f d2689f,12371.083713,14146.300572,3994.040924,13955.411987,12916.569159,12134.962767,11616.805382,12365.190366,12010.883245,10913.532571,9918.024556,9141.248604,NaN,NaN
1,cf33cb8a,cf33cb8a,d5580f74,76a16657,9ac1420f,5.367156,0.000000,58.617761,435.456412,322.485383,253.958847,233.769454,252.992992,284.516555,274.742251,41.468587,246.152500,NaN,NaN
2,cf33cb8a,cf33cb8a,d5580f74,76a16657,9b395bc9,120.462831,569.442204,226.753564,478.519945,548.853073,564.864503,458.804125,494.690402,432.012780,380.752349,493.309441,442.527260,NaN,NaN
3,cf33cb8a,cf33cb8a,d5580f74,29ded6f4,2f614c0b,23208.879752,22340.062006,1600.464821,21219.954457,22985.774451,23339.999720,19351.403126,22098.474846,22122.077469,21993.692529,20269.221852,22206.168757,NaN,NaN
4,cf33cb8a,cf33cb8a,d5580f74,29ded6f4,043b305e,4386.088856,4038.953896,186.762991,3757.329474,4340.204936,4265.247167,2957.583498,3792.334088,4064.682751,3756.061936,3999.688716,4103.523887,NaN,NaN


## Data Analysis

In [8]:
#df_dict.keys()

dict_keys(['2017', '2016', '2013', '2019', '2015', '2014', '2018', '2020'])

In [11]:
January= df_dict['2020'].columns.get_loc("enero")
df_dict['2020']['Total_Prod']=df_dict['2020'].iloc[:,January::].fillna(0).sum(axis=1)

In [12]:
df_dict['2020'].isnull().sum()

0
departamento    0
municipio       0
operadora       0
contrato        0
campo           0
enero           0
febrero         0
marzo           0
abril           0
mayo            0
junio           0
julio           0
agosto          0
Total_Prod      0
dtype: int64

In [13]:
#Eliminar duplicados, si los hay
df_dict['2020']=df_dict['2020'].drop_duplicates()

In [15]:
Prod_2020=df_dict['2020'].groupby('campo')['Total_Prod'].sum().reset_index(name ='Anual_Prod')
Highest_Prod=Prod_2020.sort_values(by=['Anual_Prod'], ascending=False).iloc[0:20,:]
display(Highest_Prod.iloc[0:5].reset_index(drop=True))
Highest_Prod.plot(x='campo', y='Anual_Prod', kind="bar",figsize=(15,5))

,campo,Anual_Prod
313,rubiales,857001.07
73,castilla,537068.14
93,chichimene,388023.81
75,castilla norte,380005.77
298,quifa,304783.67


In [16]:
Op_Field_2018=df_dict['2018'].groupby(['operadora','departamento'])['campo'].count().reset_index(name ='#Prod_Fields')
Op_casanare_2018=Op_Field_2018[(Op_Field_2018['departamento']=='CASANARE')&(Op_Field_2018['#Prod_Fields']>5)]
display(Op_casanare_2018.count())
display(Op_casanare_2018.sort_values(by=['#Prod_Fields'], ascending=False).reset_index(drop=True))
Op_casanare_2018.sort_values(by=['#Prod_Fields'], ascending=False).plot(x='operadora', y='#Prod_Fields', kind="bar",figsize=(15,5))

operadora       0
departamento    0
#Prod_Fields    0
dtype: int64

,operadora,departamento,#Prod_Fields


In [17]:
Jan_2018= df_dict['2018'].columns.get_loc("enero")
df_dict['2018']['Total_Prod']=df_dict['2018'].iloc[:,Jan_2018::].fillna(0).sum(axis=1)
Cont_Highest_Prod= df_dict['2018'].groupby('contrato')['Total_Prod'].sum().reset_index(name ='Anual_Prod')
Five_Hg_prod=Cont_Highest_Prod.sort_values(by=['Anual_Prod'], ascending=False).iloc[0:5]
display(Five_Hg_prod.reset_index(drop=True))
Five_Hg_prod.plot(x='contrato', y='Anual_Prod', kind="bar",figsize=(15,5))

,contrato,Anual_Prod
44,cubarral,2.177189e+06
138,rubiales,1.433612e+06
91,lla 34,7.255796e+05
130,quifa,5.544383e+05
75,la cira infantas,5.424151e+05


In [18]:
Op_High_Prod_2019= df_dict['2019'].groupby('operadora')['agosto'].sum().reset_index(name ='Prod_Aug')
Op_Hh_Pd_2019=Op_High_Prod_2019.sort_values(by=['Prod_Aug'], ascending=False).iloc[0:10]
display(Op_Hh_Pd_2019.reset_index(drop=True))
Op_Hh_Pd_2019.plot(x='operadora', y='Prod_Aug', kind="bar",figsize=(15,5))

,operadora,Prod_Aug
7,ecopetrol s.a.,473986.41
10,frontera energy colombia corp sucursal colombia,81441.68
11,geopark colombia s.a.s.,69154.42
24,occidental de colombia llc,53299.12
9,equion energía limited,34950.01
13,gran tierra energy colombia ltd,29555.20
21,mansarovar energy colombia ltd,25256.75
14,hocol s.a.,19614.02
28,parex resources colombia ltd. sucursal,15727.97
3,cepsa colombia s.a.,12702.09


### <center> Comparative Analysis </center> 

In [ ]:
Tri_one_2019= df_dict['2019'].iloc[:,0:8]
Tri_one_2019= Tri_one_2019.copy()
Tri_one_2020= df_dict['2020'].iloc[:,0:8]
Tri_one_2020=Tri_one_2020.copy()
Tri_two_2019= pd.concat([df_dict['2019'].iloc[:,0:5], df_dict['2019'].iloc[:,8:11]], axis=1)
Tri_two_2019=Tri_two_2019.copy()
Tri_two_2020= pd.concat([df_dict['2020'].iloc[:,0:5], df_dict['2020'].iloc[:,8:11]], axis=1)
Tri_two_2020=Tri_two_2020.copy()
Tri_one_2019['total']= Tri_one_2019.iloc[:,5::].sum(axis=1)
Tri_one_2020['total']= Tri_one_2020.iloc[:,5::].sum(axis=1)
Tri_two_2019['total']= Tri_two_2019.iloc[:,5::].sum(axis=1)
Tri_two_2020['total']= Tri_two_2020.iloc[:,5::].sum(axis=1)